In [1]:
import numpy as np
from statsmodels.discrete.discrete_model import NegativeBinomial
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
from statsmodels.discrete.truncated_model import TruncatedLFNegativeBinomialP

def fit_mle_dispersions(_counts: np.ndarray,
                        _truncated: bool
) -> np.ndarray:
    
    def _alpha_fit_func(_c, _t):
    
        _c_p = _c[_c > 0]
        if _t:
            _nb = TruncatedLFNegativeBinomialP(_c_p, np.ones(len(_c_p))).fit(disp=False, maxiter=1000, methid='ncg')
        else:
            _nb = NegativeBinomial(_c_p, np.ones(len(_c_p))).fit(disp=False)

        return _nb.params[1]
    
    
    _disp = [_alpha_fit_func(_cnt, _truncated) for _cnt in _counts.T]
    
            
    return np.array(_disp)


def fit_zmle_dispersions(_counts: np.ndarray,) -> np.ndarray:
    
    def _alpha_fit_func(_c):
    
        _nb = ZeroInflatedNegativeBinomialP(_c, np.ones(len(_c),)).fit(disp=False, method='lbfgs', maxiter=100)

        return _nb.params[-1]
    
    
    _disp = [_alpha_fit_func(_cnt) for _cnt in _counts.T]
    
            
    return np.array(_disp)
    

In [ ]:
n_gene, n_sample = 200, 100
cnts = np.random.randn(n_sample, n_gene)
alpha_mle_t = fit_mle_dispersions(cnts, _truncated=True)
alpha_mle = fit_mle_dispersions(cnts, _truncated=False)
alpha_zmle = fit_zmle_dispersions(cnts)

In [4]:
mae = lambda x, gt_x: np.abs(x - gt_x).mean().round(2)
gt_alpha = 10
print('MAE mle: ', mae(alpha_mle, gt_alpha))
print('MAE mle nb-truncated: ', mae(alpha_mle_t, gt_alpha))
print('MAE mle zinb: ', mae(alpha_zmle, gt_alpha))

MAE mle:  10.0
MAE mle nb-truncated:  692.92
MAE mle zinb:  37775.24
